In [1]:
import pandas as pd
#pd.read_csv('ks-projects-201612.csv')
#file1=pd.read_csv('ks-projects-201612.csv', engine='python')
file2=pd.read_csv('ks-projects-201801.csv', engine='python')
#pd.concat([file1,file2])
print((file2.columns))
#print(len(file1['name '].unique()))
#len(file1)
#file2.groupby(['name','launched']).count().sort_values('ID', ascending=False)#['category '].apply(lambda#.value_counts()
file2.head()

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'usd_pledged_real', 'usd_goal_real'],
      dtype='object')


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [71]:
import requests as r
import json
headers = {'Accept':'application/json, text/javascript','User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
def find_project(project_id,term):
    counter = 0
    rs = r.get(f'https://www.kickstarter.com/discover/advanced?google_chrome_workaround&'
               f'term={term}&woe_id=0&sort=magic&seed=2570673&page={counter}'
               ,headers=headers)
    data = json.loads(rs.text)
    proj_filter = list(filter(lambda x: x['id'] == project_id, data['projects']))
    if len(proj_filter) == 1:
            a = proj_filter[0]
            del a['creator']
            del a['photo']
            del a['location']
            del a['profile']
            del a['category']
            return a
    while data['has_more'] and counter < 100:
        counter +=1
        rs = r.get(f'https://www.kickstarter.com/discover/advanced?google_chrome_workaround&'
               f'term={term}&woe_id=0&sort=magic&seed=2570673&page={counter}'
               ,headers=headers)
        data = json.loads(rs.text)
        proj_filter = list(filter(lambda x: x['id'] == project_id, data['projects']))
        if len(proj_filter) == 1:
            a = proj_filter[0]
            del a['creator']
            del a['photo']
            del a['location']
            del a['profile']
            del a['category']
            return a
    return None


In [111]:
def find_project_by_id(ID):
    project_id, term = ID, file2[file2['ID']==ID].name.values
    counter = 0
    rs = r.get(f'https://www.kickstarter.com/discover/advanced?google_chrome_workaround&'
               f'term={term}&woe_id=0&sort=magic&seed=2570673&page={counter}'
               ,headers=headers)
    try:
        data = json.loads(rs.text)
    except Exception as e:
        return {'error':str(e)}, ID
    proj_filter = list(filter(lambda x: x['id'] == project_id, data['projects']))
    if len(proj_filter) == 1:
            a = proj_filter[0]
            a.pop('creator',None)
            a.pop('photo',None)
            a.pop('location',None)
            a.pop('profile',None)
            a.pop('category',None)
            return a, index
    while data['has_more'] and counter < 10:
        counter +=1
        rs = r.get(f'https://www.kickstarter.com/discover/advanced?google_chrome_workaround&'
               f'term={term}&woe_id=0&sort=magic&seed=2570673&page={counter}'
               ,headers=headers)
        try:
            data = json.loads(rs.text)
        except Exception as e:

            return {'error':str(e)}, ID
        
        proj_filter = list(filter(lambda x: x['id'] == project_id, data['projects']))
        if len(proj_filter) == 1:
            a = proj_filter[0]
            a.pop('creator',None)
            a.pop('photo',None)
            a.pop('location',None)
            a.pop('profile',None)
            a.pop('category',None)
            return a, ID,
    return None, ID


In [112]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from tqdm import tqdm_notebook as tqdm
import json
from time import time

In [121]:
#newids=[ID for ID in file2.ID.values if ID not in ids]
newids=file2[~file2.ID.isin(ids)].ID.values

In [182]:
done_res = []
done_indexes = []
done_nones = []
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = executor.map(find_project_by_id, newids)
    count = 0
    for res, index in tqdm(futures,total=len(newids)):
        done_res += [res]
        done_indexes.append(index)
        count +=1
        if count == 1000:
            with open(f'processed/{int(time())}.json','w') as f:
                json.dump(done_res,f)
                done_res = []
            count = 0

        

In [38]:
import os
len(os.listdir('processed'))

215

In [183]:
import json

ids = []
files=[]
for d in os.listdir('processed'):
    with open(f'processed/{d}') as f:
        js = json.load(f)
        files+=js
        ids += [x['id'] for x in js if x is not None and 'id' in x]


In [223]:
extra_pd=pd.DataFrame.from_records([file for file in files if file is not None])
print(len(extra_pd))
#extra_pd.drop_duplicates()
len(extra_pd)
type(extra_pd)

415989


pandas.core.frame.DataFrame

In [245]:
print(len(file2))
print(len(extra_pd))
#extra_pd.set_index('id', inplace=True)
complete=file2.merge(extra_pd,left_index=True,right_index=True)

378661
415989


In [286]:
len(complete)

354653

In [310]:
for i in range(0,len(complete),5000):
    
    complete.iloc[i:i+5000].to_csv(f'merged data {i}.csv')


In [319]:
import numpy as np
pd.read_csv('merged data 5000.csv')

,Unnamed: 0,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,staff_pick,blurb
0,30582865,Guys With Girl Problems Webseries,Webseries,Film & Video,USD,2011-03-14,5000.0,2010-12-21 21:20:55,10.00,failed,1,US,10.00,10.00,5000.00,False,"Join Zack, Tyler, Danny, Riley, Beth and Heath..."
1,30587876,Queer Street Tarot Deck,Publishing,Publishing,USD,2017-07-01,6000.0,2017-05-22 01:21:53,955.00,failed,26,US,357.00,955.00,6000.00,False,Queer Street Tarot Deck is a project of my exp...
2,30591149,Educational birthday gifts,Web,Technology,AUD,2017-09-13,3000.0,2017-07-15 06:42:46,1.00,failed,1,AU,0.00,0.80,2406.93,False,Ever thought of offering a birthday or Christm...
3,30599075,LOCAL HONEY ALBUM AND DOCUMENTARY,Documentary,Film & Video,USD,2014-09-19,16000.0,2014-07-31 21:12:43,17679.87,successful,134,US,17679.87,17679.87,16000.00,False,Local Honey is a collaborative album and docum...
4,30606080,Motown-Inspired Worship EP // Ryan Rodrigues,Music,Music,GBP,2017-08-07,10000.0,2017-06-08 16:32:17,10055.00,successful,45,GB,0.00,13050.65,12979.26,False,I'm raising £10K to record an EP of original m...
5,30612527,"Paul Smith - Album of Original Songs ""Light of...",Country & Folk,Music,USD,2011-08-17,1200.0,2011-06-18 16:08:33,75.00,canceled,2,US,75.00,75.00,1200.00,False,I'm recording a demo album of 12 original son...
6,30615342,Magnetykes,Product Design,Design,USD,2017-11-24,25000.0,2017-10-10 17:24:27,2085.00,failed,12,US,51.00,2085.00,25000.00,False,Magnetykes are a line of durable magnetic plas...
7,30616786,Aghast - A Journal of the Darkly Fantastic,Periodicals,Publishing,GBP,2014-04-29,1000.0,2014-03-28 00:54:34,1468.00,successful,106,GB,2427.43,2477.39,1687.59,True,"Aghast is a brand new, bi-annual, illustrated ..."
8,30617586,Modern Trek,Documentary,Film & Video,USD,2015-07-05,1000.0,2015-06-05 20:56:07,0.00,failed,0,US,0.00,0.00,1000.00,False,Modern Trek is a no-bull documentary providing...
9,30621218,"Run Away, Join the Circus - A Holocaust Memoir",Experimental,Theater,USD,2017-04-21,5400.0,2017-03-16 20:14:05,10312.00,successful,191,US,505.00,10312.00,5400.00,False,A theater show with circus acts telling the tr...
